In [1]:
import codecs
import re
import pandas as pd
import numpy as np
from yahooquery import Ticker
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import date, timedelta
import altair as alt

In [4]:
link = ("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies#S&P_500_component_stocks")
sp500_raw = pd.read_html(link, header=0)[0]
sp500 = [str(stock) for stock in sp500_raw["Symbol"]]

today = date.today()
start_date = date.today() - timedelta(weeks=27)
start_date = f"{start_date.year}-{start_date.month}-{start_date.day}"
today = f"{today.year}-{today.month}-{today.day}"
momentum_start = date.today() - timedelta(weeks=4)
momentum_start = f"{momentum_start.year}-{momentum_start.month}-{momentum_start.day}"

In [5]:
def compass(stocks):
    _data_points = ["Stock","Momentum", "Levy"]
    updated_stocks = []
    for stock in stocks:
        if len(pd.DataFrame(yf.Ticker(stock).history(start=start_date))["Close"]) != 0:
            updated_stocks.append(stock)
        else:
            pass
    data = pd.DataFrame({"Stock": updated_stocks, "Momentum": None, "Levy": None}, index=updated_stocks)
    for stock in updated_stocks:
        data.loc[stock]["Stock"] = pd.DataFrame(sp500_raw).set_index(sp500_raw["Symbol"]).loc[stock]["Security"]
        data.loc[stock]["Momentum"] = momentum(stock)
        data.loc[stock]["Levy"] = relative_strength(stock)
    return data
      
def relative_strength(stock, start=start_date):
    constructor = pd.DataFrame(yf.Ticker(stock).history(start=start))
    strength = constructor["Close"][-1] / np.mean(constructor["Close"][:-1])
    return strength

def momentum(stock, start=momentum_start):
    constructor = pd.DataFrame(yf.Ticker(stock).history(start=start))
    moment = constructor["Close"][-1] / constructor["Close"][0]
    return moment

In [ ]:
data = compass(sp500)

Got error from yahoo api for ticker BRK.B, Error: {'code': 'Not Found', 'description': 'No data found, symbol may be delisted'}
- BRK.B: No timezone found, symbol may be delisted
BF.B: No data found for this date range, symbol may be delisted


In [ ]:
base = alt.Chart(data)
xscale = alt.Scale(domain=(0.5, 1.5))
yscale = alt.Scale(domain=(0.5, 1.5))

points = base.mark_circle(color="red").encode(
    alt.X("Levy:Q").scale(xscale),
    alt.Y("Momentum:Q").scale(yscale),
)

text = points.mark_text(align='left',baseline='middle',dx=10).encode(
    text="Stock",
)

x_line = alt.Chart().mark_rule(strokeDash=[10, 10]).encode(y=alt.datum(1))
y_line = alt.Chart().mark_rule(strokeDash=[10, 10]).encode(x=alt.datum(1))


(points+text+x_line+y_line).properties(width=650,height=500, title="Trending Stock Indicator").interactive().save("Trendcompass.html")

In [ ]:
data["Mean"] = np.mean([data["Momentum"], data["Levy"]], axis=0)
new_data = data.sort_values(by=["Mean"], axis=0, ascending=False)[:30]

main_chart = alt.Chart(new_data, title="List of Best 30 Stocks").mark_point(color="red").encode(
alt.X("Mean:Q")
    .title("Levy: blue, Mean: red, Momentum: green")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .title("")
    .sort("-x")
    .axis(grid=True),

)

chart_1= alt.Chart(new_data).mark_point(color="green").encode(
alt.X("Momentum:Q")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .sort()
    .axis(grid=True),

)

chart_2 = alt.Chart(new_data).mark_point(color="blue").encode(
alt.X("Levy:Q")
    .scale(zero=False)
    .axis(grid=False),
alt.Y("Stock:N")
    .sort()
    .axis(grid=True),

)
(main_chart+chart_1+chart_2).properties(height=alt.Step(20)).configure_view(stroke="transparent").save("Scoreboard.html")

In [ ]:
scatter = codecs.open("Trendcompass.html", "r").read()
board = codecs.open("Scoreboard.html", "r").read()

In [ ]:
start = "var spec = {"
end = "var embedOpt = "
scatter = "".join(scatter.split(start)[1].split(end)[0]).removesuffix("};\n      ")
board = "".join(board.split(start)[1].split(end)[0]).removesuffix("};\n      ")


In [ ]:
template = codecs.open("_layouts/backup.html", "r").read()
default = template.replace("#scatterplot", scatter)
default = default.replace("#leaderboard", board)


In [ ]:
with open("_layouts/default.html", "w") as file:
    file.write(default)
